In [4]:
import hnswlib
import numpy as np
import numpy as np
import h5py
import os
import requests
import tempfile
import time
import struct
import itertools

from core_utils import read_float_binary_file,read_i8_binary_file,read_u8_binary_file,write_float_binary_file,read_int32_binary_file,compute_recall



In [5]:

[nb, nd, dataset] = read_float_binary_file('/home/rakri/arxiv/wikipedia_large/wikipedia_base_100K.bin')
[nq, ndq, queries] = read_float_binary_file('/home/rakri/arxiv/wikipedia_large/wikipedia_query.bin')
[nqgt, nkgt, gt] = read_int32_binary_file('/home/rakri/wiki100k_gs100.bin')

#names and parameters
dataset_name='wiki100'
Mb = 16
efb = 200
index_name = '/home/rakri/indices/hnsw_wiki_'+dataset_name+'M='+str(Mb)+'_efC='+str(efb) 

# for hnsw, it is only ef_search
search_params = [(10), (20), (30), (40), (50), (60), (70), (80), (90), (100)]

In [6]:
# Create an HNSW index
index = hnswlib.Index(space='l2', dim=nd)
index.init_index(max_elements=nb, M = Mb, ef_construction = efb, random_seed = 100)

if os.path.isfile(index_name):
    print("Index exists, so loading")
    index = hnswlib.Index(space='l2', dim=nd)
    index.load_index(index_name, max_elements = nb)
else:
    index.add_items(dataset)
    index.save_index(index_name)

Index exists, so loading


In [7]:
# Search for the nearest neighbor of a query vector

index.set_num_threads(1)
print("ef\tRecall\t\ttime")
for param in search_params:
    (ef_search) = param
    index.set_ef(ef_search)
    start = time.time()
    neighbors, distances = index.knn_query(queries, k=10)
    end = time.time()
    recall = compute_recall(neighbors, gt[:,:10])
    print(ef_search,"\t",recall,"\t",1000000*(end-start)/nq)
# Print the nearest neighbor
#print(neighbors)
#print(distances)

ef	Recall		time


10 	 0.68444 	 61.86943054199219
20 	 0.73324 	 114.05119895935059
30 	 0.82468 	 162.32924461364746
40 	 0.88636 	 207.39431381225586
50 	 0.9136 	 247.88260459899902
60 	 0.93184 	 286.47656440734863
70 	 0.95938 	 324.2431640625
80 	 0.98924 	 362.2021198272705
90 	 0.99904 	 401.24807357788086
100 	 0.99988 	 439.23306465148926
